# Resume NER
## Extract Information from Resumes using NER (Named Entity Recognition)

### Part 2 - NER with Spacy
In this second part of the challenge, we will be using the preprocessed data from part one to start training NER models. We will be using spacy (https://spacy.io/) here to "get our feet wet" with NER, as training spacy can be reasonably done on our laptops and does not yet necessarily require a GPU. Spacy is a powerful, effective, and resource-efficient NLP library - It might surprise us with its performance on the challenge!

We will run spacy's pretrained models on our data to get a feel for NER, and then we will perform some additional preprocessing on our data before we start training our own NER model using the labelled entities we have identified in part one. 
We will also explore evaluation metrics for NER, and decide how we want to quantify the performance of our trained models. 

* *If you need help setting up python or running this notebook, please get help from the  assistants to the professor*
* *It might be helpful to try your code out first in a python ide like pycharm before copying it an running it here in this notebook*
* *For solving the programming tasks, use the python reference linked here (Help->Python Reference) as well as Web-searches.* 

##### Reload preprocessed data
Here, we will load the data we saved in part one and save it to a variable. Provide code below to load the data and store it as a list in a variable. (Hint - use 'open' and the json module)

In [2]:
## import json module
import json
path = "data/converted_resumes.json"
## TODO open file load as json and store in "resumes" variable
with open(path, "r") as f:
    resumes = json.load(f)
## TODO print length of loaded resumes list to be sure everything ok
print(len(resumes))

690


##### Take Spacy for a spin
Before we train our own NER model to recognize the resume-specific entities we want to capture, let's see how spacy's pretrained NER models do on our data. These pretrained models can't recognize our entities yet, but let's see how they do. Run the next code block to load spacy's English language model 


In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
print(nlp)

Now we get the EntityRecognizer in the loaded nlp pipeline and display the labels it supports

In [4]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('PERCENT', 'MONEY', 'PRODUCT', 'LOC', 'DATE', 'LAW', 'CARDINAL', 'EVENT', 'PERSON', 'ORDINAL', 'FAC', 'LANGUAGE', 'QUANTITY', 'WORK_OF_ART', 'NORP', 'GPE', 'TIME', 'ORG')


##### Question: What do the 'GPE', 'FAC' and 'NORP' labels stand for? (Tipp: use either the spacy.explain method, or google the spacy.io api docs) 


In [5]:
### TODO  if you choose to use spacy's 'explain' method to get the answer to the question above, provide your code here
GPE = spacy.explain("GPE")
FAC = spacy.explain("FAC")
NORP = spacy.explain("NORP")
## print description of entities using spacy explain
print("GPE description: ", GPE)
print("FAC description: ", FAC)
print("NORP description: ", NORP)

GPE description:  Countries, cities, states
FAC description:  Buildings, airports, highways, bridges, etc.
NORP description:  Nationalities or religious or political groups


As we can see, the entities are different than the entities we will train our custom model on. 
##### Question: what entities do you think this model will find in an example resume?
 Countries, cities, states, Nationalities, ORG, LANGUAGE and DATE
 

Now we will work with one of our resumes, and get spacy to tell us what entities it recognizes. Complete the code block below to get a single resume text out of our resume list. 

In [6]:
### TODO get a single resume text and print it out
restxt = resumes[0][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Afreen Jamadar
Active member of IIIT Committee in Third year
Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
I wish to use my knowledge, skills and conceptual understanding to create excellent team
environments and work consistently achieving organization objectives believes in taking initiative
and work to excellence in my work.
WORK EXPERIENCE
Active member of IIIT Committee in Third year
Cisco Networking -  Kanpur, Uttar Pradesh
organized by Techkriti IIT Kanpur and Azure Skynet.
PERSONALLITY TRAITS:
• Quick learning ability
• hard working
EDUCATION
PG-DAC
CDAC ACTS
2017
Bachelor of Engg in Information Technology
Shivaji University Kolhapur -  Kolhapur, Maharashtra
2016
SKILLS
Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)
ADDITIONAL INFORMATION
TECHNICAL SKILLS:
• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML

Extracting entities with spacy is easy with a pretrained model. We simply call the model (here 'nlp') with our text to get a spacy Document. See https://spacy.io/api/doc for more detail. 

Execute the code below to process the resume txt.

In [7]:
doc = nlp(restxt)

The doc object has a list of entities predicted by spacy 'ents'. We would like to loop through all of these entities and print their label and associated text to see what spacy predicted for this resume.

Complete the code below to do this. You will probably need to google the spacy api docs to find the solution (Tipp: look for 'Doc.ents'). Also, trying code in your ide (for example pycharm) before copying it here might help with exploring and debugging to find the solution. 

In [8]:
##TODO loop through the doc's entities, and print the label and text for each entity found. 
entities = doc.ents
print("Label    Text\n")
for entity in entities: 
    print(entity.label_, "     " + entity.text)
 

Label    Text

GPE      Jamadar
ORG      IIIT Committee
DATE      Third year
ORG      Maharashtra
ORG      IIIT Committee
DATE      Third year
PERSON      Cisco Networking -  Kanpur
ORG      Uttar Pradesh
ORG      Techkriti IIT Kanpur
PERSON      Azure Skynet
PERSON      Quick
ORG      CDAC
DATE      2017
GPE      Kolhapur
ORG      Maharashtra
DATE      2016
ORG      HTML
DATE      Less than 1 year
PERSON      Linux
DATE      Less than 1 year
ORG      MICROSOFT
PERSON      ACCESS
PERSON      C++
PERSON      Java
ORG      XML
ORG      Operating Systems
PERSON      Windows Server
DATE      2003
GPE      Linux
ORG      MS Access
ORG      MS SQL
DATE      2008
PRODUCT      Oracle 10
PRODUCT      MySql


##### Questions: What is your first impression of spacy's NER based on the results above? Does it seem accurate/powerfull?
##### Does it make many mistakes? Do some entity types seem more accurate than others? 
No it does't, there are several worng labeld entities.
entities like Date, ORG seem to be more accurate than others 

Now as a comparison, we will list the entities contained in the resume's original annotated training data (remember, the existing annotations were created by a human-annotator, and not predicted by a machine like the entities predicted above) 

Complete the code below to do the following: 
* Access the 'entities' list of the example resume you chose, loop through the entities and print them out. 
* *Tip: one entity in the list is a tuple with the following structure: (12,1222,"label") where the first element is the start index of the entity in the resume text, the second element is the end index, and the third element is the label.
* Use this Tip to print out a formatted list of entities 



In [9]:
##TODO print original entities for one resume
res = resumes
restext = resumes[0][0]
labeled_ents = resumes[0][1]['entities']
print(labeled_ents)
## TDOD print out formatted list of entity labels and text
for ent in labeled_ents:
    label = ent[2]
    start = ent[0]
    end = ent[1]
    print("{}: {}".format(label, restext[start:end]))

[[1155, 1199, 'Email Address'], [1143, 1240, 'Links'], [743, 1141, 'Skills'], [729, 733, 'Graduation Year'], [675, 703, 'College Name'], [631, 673, 'Degree'], [625, 630, 'Graduation Year'], [614, 623, 'College Name'], [606, 612, 'Degree'], [438, 454, 'Companies worked at'], [104, 148, 'Email Address'], [62, 68, 'Location'], [0, 14, 'Name']]
Email Address: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
Links: https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN
Skills: Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)

ADDITIONAL INFORMATION

TECHNICAL SKILLS:

• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML
• Operating Systems: Windows […] Windows Server 2003, Linux.
• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.
Graduation Year: 2016
College Name: Shivaji University Kolhapur 
Degree: Bachelor of 

As we already know, the annotated entities in the training data are different than the entities spacy can recognize with it's pretrainied models, so we need to train a custom NER model. We will get started with that now. 

##### Prepare Training Data for NER model training
We need to do some more preprocessing of our training data before we can train our model.

Remember the entity labels you chose in part 1 of the challenge? We will be training a model to predict those entities.
As a first step, we will gather all resumes that contain at least one training annotation for those entities.

Complete and execute the code below to gather your training data. 

In [10]:
##TODO Store the entity labels you want to train for as array in chosen_entity_labels
import numpy as np
chosen_entity_labels = np.array(["Name",  "Companies worked at", "Degree"])

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
## TODO use the gather candidates methods and store result in training_data variable
training_data = gather_candidates(resumes, chosen_entity_labels)
print(training_data[0])
print("Gathered {} training examples".format(len(training_data)))

['Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming

Now we have those training examples which contain the entities we are interested in. Do you have at least a few hundred examples? If not, you might need to re-think the entities you chose or try just one or two of them and re-run the notebooks. It is important that we have several hundred examples for training (e.g. more than 200. 3-500 is better). 

##### Remove other entity annotations from training data
Now that we have our training data, we want to remove all but relevant (chosen) entity annotations from this data, so that the model we train will only train for our entities. Complete and execute the code below to do this. 

In [11]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered               
## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable
training_data_filterd = []
for resume in training_data:
    X = filter_ents(resume[1]['entities'], chosen_entity_labels)
    resume[1]['entities'] = X
    training_data_filterd.append(resume)
training_data = training_data_filterd

##### Remove resumes that cause errors in spacy
Depending on what entities you chose, some of the resumes might cause errors in spacy. We don't need to get into details as to why, suffice to say it has to do with whitespace and syntax in the entity annotations. If these resumes are not removed from our training data, spacy will throw an exception during training, so we need to remove them first. 

We will use the remove_bad_data function below to do this. This function does the following:
* calls train_spacy_ner with debug=True and n_iter=1. This causes spacy to process the documents one-by-one, and gather the documents that throw an exception in a list of "bad docs" which it returns. 
* You will complete the function to remove any baddocs (returned by remove_bad_data) from your training data list. 

You may or may not have any bad documents depending on the entities you chose. In any case, there should not be more than a dozen or so bad docs.  

In [12]:
from spacy_train_resume_ner import train_spacy_ner

def remove_bad_data(training_data):
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    ## training data is list of lists with each list containing a text and annotations
    ## baddocs is a set of strings/resume texts.
    ## TODO complete implementation to filter bad docs and store filter result (good docs) in filtered variable
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method. It may take a few minutes for the method to complete.
## you will know it is complete when the print output above. 
X = remove_bad_data(training_data)

Created blank 'en' model
Exception thrown when processing doc:
("Neeraj Dwivedi\nSenior Sales Executive - Kansai Nerolac Paints Ltd\n\nMumbai, Maharashtra - Email me on Indeed: indeed.com/r/Neeraj-Dwivedi/8f053ed44cdef8b2\n\nWORK EXPERIENCE\n\nSenior Sales Executive\n\nKansai Nerolac Paints Ltd -  Mumbai, Maharashtra -\n\nAugust 2017 to Present\n\nIncreased the sales from -22% to +6% through existing dealers and prospecting new dealers to\nachieve the target within 6 months of joining.\n• Maintaining excellent relations with dealers to increase revenue by 10%.\n• Management of sales team (up to 3 members)\n• Achieved monthly targets amounting to 40 - 60 lakhs monthly with timely collection.\n• Maintain accounts clarity with dealers and ensure that the credit notes reach the dealers on\ntime.\n• Business development by conducting regular meetings with influencers such as architects,\nbuilding contractors, housing societies, painters.\n• Ensuring proper visibility of the products at vari

##### Question: How many bad docs did you have? What is the size of your new (filtered) training data? 
Just one bad doc.
The size is 559

##### Train/Test Split
Now before we train our model, we have to split our available training data into training and test sets. Splitting our data into train and test (or holdout) datasets is a fundamental technique in machine learning, and essential to avoid the problem of overfitting.
Before we go on, you should get a grasp of how train/test split helps us avoid overfitting. Please take the time now to do a quick web search on the topic. There are many resources available. You should search for "train test validation overfitting" or some subset of those terms.

Here are a few articles to start with:
* https://machinelearningmastery.com/a-simple-intuition-for-overfitting/
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation (Note: you are free to install scikit learn and use the train_test_split method documented here, but it is not necessary. It is the concept that is important)

##### Question: What is overfitting and how does doing a train/test split help us avoid overfitting when training our models? Please answer in your own words. 
when the model fit the data provided in training phase in such way that generate a big amount of bias when we use the model on other data. 

Now that we understand why we do a train/test split, we will write some code that splits our data into train and test sets. Usually we want around 70-80% of the data for train, and the rest for test. 
##### TODO: Complete the code below to create a train and test dataset

In [13]:
##TODO complete the implementation  of the train test split function below
from sklearn.model_selection import train_test_split
## TODO chose train size percent and call train test split, storing results in "train" and "test" variables.
train,test = train_test_split(X, test_size=0.20)
## TODO use python assert to assert that the size of train and test sets add up to the size of all the data 
assert len(train) + len(test) == len(X)

##### Train a spacy ner model with our training data
OK, now it is (finally) time to train our own custom NER model using spacy. Because our training data has been preprocessed to only include annotations for the entities we are interested in, the model will only be able to predict/extract those entities. 
*Depending on your computer, this step may take a while.* Training 20 epochs (iterations) using 480 training examples takes around 10 minutes on my machine (core i7 CPU). You will see output like *Losses {'ner':2342.23342342}* after each epoch/iteration. The default number of iterations is 20, so you will see this output 20 times. When this step is done, we will use the trained ner model to perform predictions on our test data in our test set.  

In [14]:
## run this code to train a ner model using spacy
custom_nlp,_= train_spacy_ner(train,n_iter=20)

Created blank 'en' model
Losses {'ner': 31513.902025726202}
Losses {'ner': 31575.179004938487}
Losses {'ner': 34592.16721692965}
Losses {'ner': 32170.18650860223}
Losses {'ner': 31618.599995860946}
Losses {'ner': 29235.028641246823}
Losses {'ner': 38210.04297048364}
Losses {'ner': 19330.94359249965}
Losses {'ner': 6773.520552361799}
Losses {'ner': 7342.400287367322}
Losses {'ner': 5737.908540177597}
Losses {'ner': 6017.940510004024}
Losses {'ner': 5161.601522376947}
Losses {'ner': 8244.134018287572}
Losses {'ner': 5836.783466333511}
Losses {'ner': 6321.758023662678}
Losses {'ner': 3687.7879184904555}
Losses {'ner': 4264.082984319843}
Losses {'ner': 3080.0613154434404}
Losses {'ner': 3717.232522182456}


##### Inspect NER predictions on one sample resume
Now that we have a trained model, let's see how it works on one of our resumes. 

In [15]:
## TODO fetch one resume out of our test dataset and store to the "resume" variable
resume = test[21]
## TODO create a spacy doc out of the resume using our trained model and save to the "doc" variable 
doc = custom_nlp(resume[0])

Now we will output the predicted entities and the existing annotated entities in that doc

In [16]:
## TODO output label and text of predicted entities (in "ents" variable of the spacy doc created above)
print("PREDICTED:")
for ent in doc.ents:
    print(ent.label_, ent)
    
## TODO output labeled entities (in "entities" dictionary of resume)
print("\nLABELED:")
for ent in resume[1]['entities']:
    label = ent[2]
    start = ent[0]
    end = ent[1]
    print(label, resume[0][start:end])

PREDICTED:
Name Shiksha Bhatnagar
Degree pass 12 in medical

LABELED:
Companies worked at Microsoft 
Degree pass 12 in medical
Companies worked at Microsoft 
Name Shiksha Bhatnagar


#### Evaluation Metrics for NER
Now that we can predict entities using our trained model, we can compare our predictions with the original annotations in our training data to evaluate how well our model performs for our task. The original annotations have been annotated manually by human annotators, and represent a "Gold Standard" against which we can compare our predictions. 

For most classification tasks, the most common evaluation metrics are:
* accuracy
* precision
* recall
* f1 score

In order to understand these metrics, we need to understand the following concepts:
* True positives - How many of the predicted entities are "true" according to the Gold Standard? (training annotation) 
* True negatives - How many entities did the model not predict which are actually not entities according to the Gold Standard?
* False positives - How many entities did the model predict which are NOT entities according to the Gold Standard?  
* False negatives - How many entities did the model "miss" - e.g. did not recognize as entities which are entities according to the Gold Standard? 

Before we go on, it is important that you understand true/false positives/negatives as well as the evaluation metrics above. Take some time now to research the web in order to find answers to the following questions:

##### Question: How are the evaluation metrics above defined in the context of evaluating Machine Learning models? How do they relate to True/False Positives/Negatives above? Please provide an intuitive description as well as the mathmatical formula for each metric. 
F1 Score is used to measure a test's accuracy
F1 Score: A combination of Precision and Recall
          (2 * Precision * Recall) / (Precision + Recall)
precision: The approach here is to find what percentage of the model’s positive (1’s) predictions are accurate
           What percentage of predicted 1's are correct?
           True positives/(True positives + False positives)
Recall: percentage of ALL positive (1’s) were covered.
        What percentage of all 1's were correctly predicted?
        True positives/(True positives + False negatives)

##### Calculating Metrics based on token-level annotations or full entity-level. 
The concepts above are our first step toward understanding how to evaluate our model effectively. However, in NER, we need to take into account that we can calculate our metrics either based on all tokens (words) found in the document, or only on the entities found in the document.  

##### Token-Level evaluation. 
Token level evaluation evaluates how accurately did the model tag *each individual word/token* in the input. In order to understand this, we need to understand something called the "BILUO" Scheme (or BILOU or BIO). The spacy docs have a good reference. Please read and familiarize yourself with BILUO. 

https://spacy.io/api/annotation#biluo

Up to now, we have not been working with the BILUO scheme, but with "offsets" (for example: (112,150,"Email") - which says there is an "Email" entity between positions 112 and 150 in the text). We would like to be able to evaluate our models on a token-level using BILUO - so we need to convert our data to BILUO. Fortunately, Spacy provides a helper method to do this for us.

*Execute the code below to see how our "Gold Standard" and predictions for our example doc above look in BILUO scheme.* 
Note: some of the lines might be ommited for display purposes. 

In [17]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,
                                                   [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df(custom_nlp, test[1])
display(bilou_df)  

,Tokens,Predicted,True
0,Nidhi,B-Name,B-Name
1,Pandit,L-Name,L-Name
2,,O,O
3,Test,O,O
4,Engineer,O,O
5,-,O,O
6,Infosys,B-Companies worked at,-
7,Limited,L-Companies worked at,-
8,,O,-
9,-,O,O


Based on this output, it should be very easy to calculate a token-level accuracy. We simply compare the "Predicted" to "True" columns and calculate what percentage are the same. 

In [18]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df['Predicted'] == bilou_df['True']]
## accuracy is the length of this subset divided by the length of bilou_df
accuracy =same_df.shape[0] / bilou_df.shape[0] 
print("Accuracy on one resume: ",accuracy)

Accuracy on one resume:  0.9868995633187773


The accuracy might seem pretty good... if it is not 100%, then let's print out those tokens where the model predicted something different than the gold standard by running the code below. 

Note - if your score on one doc is 100%, pick another document and re-run the last few cells above. 

In [27]:
## TODO find all rows in bilou_df where "Predicted" not equal to "True" column. 
diff_df = bilou_df[bilou_df['Predicted'] != bilou_df['True']]
display(diff_df)

,Tokens,Predicted,True
6,Infosys,B-Companies worked at,-
7,Limited,L-Companies worked at,-
8,,O,-
56,Infosys,B-Companies worked at,O
57,Limited,L-Companies worked at,O
285,Infosys,B-Companies worked at,O
286,Limited,L-Companies worked at,O
297,Infosys,B-Companies worked at,O
298,Limited,L-Companies worked at,O


Now let's calculate the accuracy on all our test resumes and average them for an accuracy score. 

Please complete the code below to report an accuracy score on our test resumes

In [20]:
import numpy as np
doc_accuracy = []
for res in test:
    ## TODO calculate accurac for each 'res' and append to doc_accuracy list 
    same_df = bilou_df[bilou_df['Predicted'] == bilou_df['True']]
    accuracy = same_df.shape[0] / bilou_df.shape[0]
    doc_accuracy.append(accuracy)

## TODO calculate mean/average of doc_accuracy (Tip: use numpy!)
total_acc = sum(doc_accuracy) / len(test)
print("Accuracy: ",total_acc)

Accuracy:  0.9868995633187756


##### Question: how does the model perform on token-level accuracy? What did it miss? In those cases where the predictions didn't match the gold standard, were the predictions plausible or just "spurious" (wrong)? 
The perfomance is nearly perfect. it miss something in Degree label most the time. i think the predictions were plausible

##### Question: What might the advantages and disadvantages be of calculating accuracy on token-level? Hint: think about a document with 1000 tokens where only 10 tokens are annotated as entities. What might the accuracy be on such a document?


##### Entity-Level evaluation #####
Another method of evaluating the performance of our NER model is to calculate metrics not on token-level, but on entity level. There is a good blog article that describes this method. 

http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The article goes into some detail, the most important part is the scenarios described in the section "Comparing NER system output and golden standard". 

##### Question: how do the first 3 scenarios described in the section "Comparing NER system output and golden standard" correlate to  true/false positives/negatives? 
Surface string and entity type match
true postivies true negatives

System hypothesized an entity
true postivies false positives

System misses an entity
true negatives false negatives


##### Precision, Recall, F1 #####

Now we would like to calculate precision, recall, and f1 for each entity type we are interested in (our chosen entities). To do this, we need to understand the formulas for each. A good article for this is https://skymind.ai/wiki/accuracy-precision-recall-f1. 

##### Question: how can we calculate precision, recall and f1 score based on the information above? Please provide the formulas for each #####
System hypothesized an entity:
precision = true positives / (true positives + false positives)

System misses an entity
recall = true posititves / (true positives + false negatives)

f1 score = 2 * ( (precision * recall) / (precision + recall) )

Now supply code below which calculates precision and recall and F1 on our test data for each entity type we are interested in. 



In [21]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for tres in test:
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 
        tres_df = make_bilou_df(custom_nlp, tres)
        same_df = tres_df[tres_df['Predicted'] == tres_df['True']]
        diff_df = tres_df[tres_df['Predicted'] != tres_df['True']]
        ## calculate true false positives and false negatives for each resume
        tp = len(same_df[same_df['Predicted'].str.contains(label)])
        fp = len(diff_df[diff_df['Predicted'].str.contains(label)]) 
        fn = len(diff_df[diff_df['True'].str.contains(label)])
        ## aggregate result for each resume to totals
        true_positives = true_positives + tp
        false_positives = false_positives + fp
        false_negatives = false_negatives + fn
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out        
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * ( (precision * recall) / (precision + recall) )
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame(data, columns=['Precision', 'Recall', 'F1'], index=chosen_entity_labels)
display(metric_df)

For label 'Name' tp: 229 fp: 2 fn: 14
Precision:  0.9913419913419913
Recall:  0.9423868312757202
F1:  0.9662447257383966
For label 'Companies worked at' tp: 1004 fp: 473 fn: 223
Precision:  0.6797562626946513
Recall:  0.8182559087204564
F1:  0.742603550295858
For label 'Degree' tp: 374 fp: 227 fn: 69
Precision:  0.6222961730449251
Recall:  0.8442437923250564
F1:  0.7164750957854407


,Precision,Recall,F1
Name,0.991342,0.942387,0.966245
Companies worked at,0.679756,0.818256,0.742604
Degree,0.622296,0.844244,0.716475


Now we compute an average score for each metric. 

In [22]:
## TODO compute average metrics And print them out. Use pandas dataframe "mean" method to do this
average_precision = metric_df['Precision'].mean()
average_recall = metric_df['Recall'].mean()
average_f1 = metric_df['F1'].mean()
print('Average Precision: ', average_precision)
print('Average Rcall: ', average_recall)
print('Average F1: ', average_f1)

Average Precision:  0.7644648090271892
Average Rcall:  0.8682955107737443
Average F1:  0.8084411239398985


##### Question: how do the average metrics here (computed on entity-level) compare to the token-level accuracy score above? Which metric(s) would you prefer to use to evaluate the quality of your model? Why? 
the average metrics here is less than the token level.
i would perfer the entity level metrics, bcause it is for me is easyer to understand and in general it is more speccific

We are almost Done with part II! We just need to save our BILUO training data for reuse in Part III. Run the code below to do this. 

In [23]:
## TODO persist BILUO data as text
print("Make bilou dfs")
training_data_as_bilou = [make_bilou_df(custom_nlp,res) for res in train]
test_data_as_bilou = [make_bilou_df(custom_nlp,res) for res in test]
print("Done!")
training_df = pd.DataFrame(columns = ["text","ner","doc","ner_spacy"])
test_df = pd.DataFrame(columns = ["text","ner","doc","ner_spacy"])
for idx,df in enumerate(training_data_as_bilou):
    df2 = pd.DataFrame()
    df2["text"] = df["Tokens"]
    df2["ner"] = df["True"]
    df2["ner_spacy"]=df["Predicted"]
    df2["doc"]=idx
    training_df = training_df.append(df2)
for idx,df in enumerate(test_data_as_bilou):
    df2 = pd.DataFrame()
    df2["text"] = df["Tokens"]
    df2["ner"] = df["True"]
    df2["ner_spacy"]=df["Predicted"]
    df2["doc"]=idx
    test_df = test_df.append(df2)

with open("data/flair/train_res_bilou.txt",'w+',encoding="utf-8") as f:
    training_df.to_csv(f,sep=" ",encoding="utf-8",index=False)
with open("data/flair/test_res_bilou.txt",'w+',encoding="utf-8") as f:
    training_df.to_csv(f,sep=" ",encoding="utf-8",index=False)

Make bilou dfs
Done!


c:\users\bassel\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Now let's load the data we persisted with flair before we go on

In [24]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
columns = {1:"ner",3:"text"}
corpus = NLPTaskDataFetcher.load_column_corpus("data/flair",column_format=columns, train_file="train_res_bilou.txt",
                                               test_file="test_res_bilou.txt")

ModuleNotFoundError: No module named 'flair'

If you could load the corpus without error, you are ready to go on to part 3, where we will work with flair nlp! 